In [1]:
import requests
import pandas as pd
from datetime import datetime
from time import sleep
from concurrent.futures import ThreadPoolExecutor, as_completed
# ------------------- Konfiguration -------------------
BASE_URL = "https://dataset.api.hub.geosphere.at/v1"
DATASET = "klima-v2-1h"
START_DATE = datetime(2014, 1, 1)

# Gewünschte Parameterliste
PARAMETERS = ["tl", "rf", "ff", "ffx", "cglo", "so_h", "rr", "rrm", "tb10", "tb20"]

# ------------------- Schritt 1: Stationen abrufen -------------------
def get_station_list():
    url = f"{BASE_URL}/station/historical/{DATASET}/metadata"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()              # Das ist ein Dict mit Schlüssel 'stations'
    stations = data["stations"]         # Hier extrahieren wir die Liste
    return [{"station_id": s["id"], "name": s["name"]} for s in stations]


def check_availability_parallel(station_id, param):
    url = f"{BASE_URL}/station/historical/{DATASET}/available-times"
    params = {"station_ids": station_id, "parameters": param}
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        if param not in data or not data[param]:
            return param, "❌"
        start_time = data[param][0]["start"]
        if datetime.fromisoformat(start_time) <= START_DATE:
            return param, "✔️"
        else:
            return param, "❌"
    except:
        return param, "❌"

def build_availability_table_fast():
    stations = get_station_list()
    availability = []

    for i, station in enumerate(stations):
        print(f"[{i+1}/{len(stations)}] Prüfe {station['name']} ({station['station_id']})...")
        row = {"station_id": station["station_id"], "name": station["name"]}
        with ThreadPoolExecutor(max_workers=20) as executor:
            futures = {executor.submit(check_availability_parallel, station["station_id"], p): p for p in PARAMETERS}
            for future in as_completed(futures):
                param, result = future.result()
                row[param] = result
        availability.append(row)

    return pd.DataFrame(availability)

# ------------------- Schritt 4: Ausführen -------------------
df = build_availability_table_fast()

# Ergebnis speichern oder anzeigen
df.to_csv("geosphere_verfuegbarkeit.csv", index=False)
df.head()


[1/811] Prüfe Aflenz (1)...
[2/811] Prüfe Aigen im Ennstal (2)...
[3/811] Prüfe Allentsteig (3)...
[4/811] Prüfe Amstetten (4)...
[5/811] Prüfe Bad Aussee (5)...
[6/811] Prüfe Bad Bleiberg (6)...
[7/811] Prüfe Bad Gastein (7)...
[8/811] Prüfe Bad Gleichenberg (8)...
[9/811] Prüfe Bad Goisern (9)...
[10/811] Prüfe Bad Mitterndorf (10)...
[11/811] Prüfe Bad Radkersburg (11)...
[12/811] Prüfe Baden (12)...
[13/811] Prüfe Bernstein (13)...
[14/811] Prüfe Bischofshofen (14)...
[15/811] Prüfe Bregenz (15)...
[16/811] Prüfe Brenner (16)...
[17/811] Prüfe Bruck an der Mur (17)...
[18/811] Prüfe Bruckneudorf (18)...
[19/811] Prüfe Dellach (19)...
[20/811] Prüfe Ehrwald (20)...
[21/811] Prüfe Bad Eisenkappel (21)...
[22/811] Prüfe Eisenstadt Nordost (22)...
[23/811] Prüfe Enns (23)...
[24/811] Prüfe Feldkirch (24)...
[25/811] Prüfe Fischbach (25)...
[26/811] Prüfe Freistadt (26)...
[27/811] Prüfe Gallspach (27)...
[28/811] Prüfe Galtür (28)...
[29/811] Prüfe Gmunden (29)...
[30/811] Prüfe Graz U

,station_id,name,so_h,cglo,tb10,rrm,rf,ffx,ff,tb20,tl,rr
0,1,Aflenz,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
1,2,Aigen im Ennstal,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
2,3,Allentsteig,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
3,4,Amstetten,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
4,5,Bad Aussee,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌


In [2]:
df


,station_id,name,so_h,cglo,tb10,rrm,rf,ffx,ff,tb20,tl,rr
0,1,Aflenz,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
1,2,Aigen im Ennstal,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
2,3,Allentsteig,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
3,4,Amstetten,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
4,5,Bad Aussee,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
...,...,...,...,...,...,...,...,...,...,...,...,...
806,20902,Bad Radkersburg,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
807,20903,Bad Radkersburg,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
808,21101,Loibl Tunnel,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌
809,21300,Bad Eisenkappel,❌,❌,❌,❌,❌,❌,❌,❌,❌,❌


In [7]:
station_id = 1  # Aflenz
parameter = "tl"
url = f"https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1h/available-times"
params = {"station_ids": station_id, "parameters": parameter}

response = requests.get(url, params=params)
print(response.status_code)
print(response.json())

404
{'detail': 'Not Found'}


In [8]:
url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1h/available-times"
params = {"station_ids": 105, "parameters": "tl"}  # Wien Hohe Warte

r = requests.get(url, params=params)
print(r.status_code)
print(r.json())

404
{'detail': 'Not Found'}


In [12]:
import requests
from datetime import datetime

# Basis-Parameter
station_id = 1
base_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1h"
start = "2014-01-01T00:00"
end = "2014-01-01T01:00"  # kurzes Zeitfenster
parameters = ["tl", "rf", "ff", "ffx", "cglo", "so_h", "rr", "rrm", "tb10", "tb20"]

# Ergebnisse prüfen
verfuegbarkeit = {}

for param in parameters:
    params = {
        "station_ids": station_id,
        "parameters": param,
        "start": start,
        "end": end
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        try:
            data = response.json()
            print(data)
            values = data.get("data", {}).get(str(station_id), {}).get(param, [])
            verfuegbarkeit[param] = "✔️" if any(values) else "❌"
        except Exception:
            verfuegbarkeit[param] = "❌"
    else:
        verfuegbarkeit[param] = "❌"

# Ergebnis anzeigen
print(f"Verfügbarkeit für Station {station_id} ab {start}:")
for param, status in verfuegbarkeit.items():
    print(f"{param}: {status}")

{'media_type': 'application/json', 'type': 'FeatureCollection', 'version': 'v1', 'timestamps': ['2014-01-01T00:00+00:00', '2014-01-01T01:00+00:00'], 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [47.54594, 15.24069]}, 'properties': {'parameters': {'tl': {'name': 'Lufttemperatur 2m', 'unit': '°C', 'data': [0.6, 0.8]}}, 'station': 1}}]}
{'media_type': 'application/json', 'type': 'FeatureCollection', 'version': 'v1', 'timestamps': ['2014-01-01T00:00+00:00', '2014-01-01T01:00+00:00'], 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [47.54594, 15.24069]}, 'properties': {'parameters': {'rf': {'name': 'Relative Feuchte', 'unit': '%', 'data': [96, 93]}}, 'station': 1}}]}
{'media_type': 'application/json', 'type': 'FeatureCollection', 'version': 'v1', 'timestamps': ['2014-01-01T00:00+00:00', '2014-01-01T01:00+00:00'], 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [47.54594, 15.24069]}, 'properties': {'

In [13]:
import requests

station_id = 1
base_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v2-1h"
start = "2014-01-01T00:00"
end = "2014-01-01T01:00"
parameters = ["tl", "rf", "ff", "ffx", "cglo", "so_h", "rr", "rrm", "tb10", "tb20"]

verfuegbarkeit = {}

for param in parameters:
    params = {
        "station_ids": station_id,
        "parameters": param,
        "start": start,
        "end": end
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        try:
            data = response.json()
            features = data.get("features", [])
            if features:
                values = features[0]["properties"]["parameters"].get(param, {}).get("data", [])
                verfuegbarkeit[param] = "✔️" if any(v is not None for v in values) else "❌"
            else:
                verfuegbarkeit[param] = "❌"
        except Exception as e:
            print(f"Fehler bei {param}: {e}")
            verfuegbarkeit[param] = "❌"
    else:
        verfuegbarkeit[param] = "❌"

print(f"Verfügbarkeit für Station {station_id} ab {start}:")
for param, status in verfuegbarkeit.items():
    print(f"{param}: {status}")


Verfügbarkeit für Station 1 ab 2014-01-01T00:00:
tl: ✔️
rf: ✔️
ff: ✔️
ffx: ✔️
cglo: ✔️
so_h: ✔️
rr: ✔️
rrm: ❌
tb10: ❌
tb20: ❌
